In [1]:
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
tf.set_random_seed(777)

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
# hyper parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100

In [26]:
class Model :
    def __init__(self, sess, name) :
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self) :
        with tf.variable_scope(self.name) :
            
            self.training = tf.placeholder(tf.bool)
            
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            # Layer1
            conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, 
                                     kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout1 = tf.layers.dropout(inputs = pool1, rate = 0.3, training = self.training)
            
            # Layer2
            conv2 = tf.layers.conv2d(inputs = dropout1, filters = 64, kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout2 = tf.layers.dropout(inputs = pool2, rate = 0.3, training = self.training)
            
            # Layer3
            conv3 = tf.layers.conv2d(inputs = dropout2, filters= 128, kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size = [2, 2], padding = 'SAME', strides = 2)
            dropout3 = tf.layers.dropout(inputs = pool3, rate = 0.3, training = self.training)
            
            # Dense Layer with Relu
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs = flat, units = 625, activation = tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs = dense4, rate = 0.5, training = self.training)
            
            # Logits (no activation) Layer
            self.logits = tf.layers.dense(inputs = dropout4, units = 10)
            
        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def train(self, x_data, y_data, training = True) :
        return self.sess.run([self.cost, self.optimizer], feed_dict = {self.X : x_data, self.Y : y_data, self.training : training})
    
    def predict(self, x_test, training = False) :
        return self.sess.run(self.logits, feed_dict = {self.X : x_test, self.training : training})
    
    def get_accuracy(self, x_test, y_test, training = False) :
        return self.sess.run(self.accuracy, feed_dict = {self.X : x_test, self.Y : y_test, self.training : training})

In [27]:
tf.reset_default_graph()

In [28]:
# initialize
sess = tf.Session()

models = []
num_models = 2
for m in range(num_models) :
    models.append(Model(sess, "model"+str(m)))
# print(models)

In [29]:
sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs) :
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch) :
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        # train each model
        for m_idx, m in enumerate(models) :
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)
    
print('Learning Finished!')

Learning Started!
Epoch: 0001 cost = [0.27920389 0.27820183]
Epoch: 0002 cost = [0.08658006 0.08701599]
Epoch: 0003 cost = [0.06789637 0.06804895]
Epoch: 0004 cost = [0.0562183  0.05426845]
Epoch: 0005 cost = [0.05184626 0.05000754]
Epoch: 0006 cost = [0.04414639 0.04671458]
Epoch: 0007 cost = [0.03975264 0.04049615]
Epoch: 0008 cost = [0.03781174 0.0387194 ]
Epoch: 0009 cost = [0.03531226 0.03632418]
Epoch: 0010 cost = [0.03329245 0.03387006]
Epoch: 0011 cost = [0.03121701 0.03087787]
Epoch: 0012 cost = [0.02896714 0.0326848 ]
Epoch: 0013 cost = [0.02918668 0.02885621]
Epoch: 0014 cost = [0.02814424 0.02729278]
Epoch: 0015 cost = [0.02699725 0.02767275]
Epoch: 0016 cost = [0.02564023 0.025902  ]
Epoch: 0017 cost = [0.0244164  0.02463543]
Epoch: 0018 cost = [0.02335709 0.02405535]
Epoch: 0019 cost = [0.02327606 0.02499528]
Epoch: 0020 cost = [0.02189595 0.02200428]
Learning Finished!


In [30]:
# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models) :
    print(m_idx, 'Accuracy : ', m.get_accuracy(mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    print('p : ', p)
    predictions += p
    
ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy : ', sess.run(ensemble_accuracy))

0 Accuracy :  0.9948
p :  [[-16.83276     -4.3433867   -6.305209   ...  17.781023    -9.463876
   -0.38113874]
 [  0.08764616  -3.8498147   18.129852   ...  -8.517834    -6.573365
  -10.709008  ]
 [ -7.5381103   11.227948    -8.901385   ...  -2.851072    -2.3880227
   -8.338399  ]
 ...
 [-19.449745    -5.48444    -13.535196   ...  -9.380554    -2.2992592
   -3.6047192 ]
 [ -8.767304   -11.428932   -15.997544   ... -10.307148     3.0833423
   -7.2274513 ]
 [ -3.6613443   -9.346063    -7.339178   ... -13.949722    -2.2441142
  -12.740422  ]]
1 Accuracy :  0.9944
p :  [[-10.293297    -5.6503806   -4.9163756  ...  15.143787    -5.0571413
    0.47687152]
 [ -2.4508305   -1.6520817   21.44017    ...  -4.9495535  -11.431305
   -7.2280774 ]
 [ -5.372742    10.00151     -7.307794   ...  -4.0393357   -1.9693264
   -7.3044972 ]
 ...
 [-19.944275    -5.9947953  -12.991423   ...  -8.954208    -3.872554
   -4.437444  ]
 [-12.038458   -13.284518   -15.975221   ... -16.863405     2.5544565
   -7.07587

In [32]:
predictions

array([[-27.12605667,  -9.99376726, -11.2215848 , ...,  32.92481041,
        -14.52101707,   0.09573278],
       [ -2.3631843 ,  -5.50189638,  39.57002258, ..., -13.4673872 ,
        -18.00467014, -17.93708563],
       [-12.91085243,  21.22945786, -16.2091794 , ...,  -6.8904078 ,
         -4.35734904, -15.64289618],
       ...,
       [-39.39402008, -11.47923517, -26.52661896, ..., -18.33476257,
         -6.17181325,  -8.04216337],
       [-20.80576229, -24.71345043, -31.97276497, ..., -27.17055321,
          5.63779879, -14.30332899],
       [ -6.70444393, -20.16007328, -15.32414532, ..., -32.69601059,
         -5.41345906, -25.99681091]])

In [33]:
# tf.argmax(predictions, 1)

<tf.Tensor 'ArgMax_6:0' shape=(10000,) dtype=int64>

In [31]:
# ensemble_correct_prediction

<tf.Tensor 'Equal_2:0' shape=(10000,) dtype=bool>

In [25]:
# test_size = len(mnist.test.labels)
# print(test_size)
# predictions = np.zeros([test_size, 10])
# predictions += 3
# print(predictions)